In [77]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [78]:
df = pd.read_csv('data2.csv',encoding = 'shift-jis')
df.head(10)

,,,,,,,,,,ダウンロードした時刻：2021/06/08 02:30:02
NaN,東京,東京,東京,東京,東京,東京,東京,東京,東京,東京
年月日,平均気温(℃),平均気温(℃),平均気温(℃),降水量の合計(mm),降水量の合計(mm),降水量の合計(mm),降水量の合計(mm),平均雲量(10分比),平均雲量(10分比),平均雲量(10分比)
NaN,NaN,品質情報,均質番号,NaN,現象なし情報,品質情報,均質番号,NaN,品質情報,均質番号
2010/1/1,4.8,8,1,0,1,8,1,0.0,8,1
2010/1/2,6.3,8,1,0,1,8,1,0.0,8,1
2010/1/3,5.7,8,1,0,1,8,1,6.3,8,1
2010/1/4,6.5,8,1,0,1,8,1,5.0,8,1
2010/1/5,7.3,8,1,2.5,0,8,1,3.8,8,1
2010/1/6,6.5,8,1,0,1,8,1,0.5,8,1
2010/1/7,7.3,8,1,0,1,8,1,6.3,8,1


In [79]:
df.shape, df.columns

((3656, 1), Index(['ダウンロードした時刻：2021/06/08 02:30:02'], dtype='object'))

In [80]:
#df.info(),df.columns

In [81]:
df = pd.read_csv('data2.csv',encoding = 'shift-jis')
df = df.reset_index()
df = df.drop([0,2]).reset_index(drop = True)
df = df.loc[:,['level_0','level_1','level_4','level_8']]
#df.columns = df.iloc[0,:]
df.columns = ['date','temperture','rain','cloud']
df = df.drop([0]).reset_index(drop = True)

df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
df[df.columns[1]] = pd.to_numeric(df[df.columns[1]])
df[df.columns[2]] = pd.to_numeric(df[df.columns[2]])
df[df.columns[3]] = pd.to_numeric(df[df.columns[3]])
df['POSIX'] = df[df.columns[0]].astype(np.int64).values//10**9

df['year'] = df[df.columns[0]].dt.year
df['month'] = df[df.columns[0]].dt.month
df['day'] = df[df.columns[0]].dt.day
df['day_of_week'] = df[df.columns[0]].dt.dayofweek

day_ago = 7

for i in range(1,day_ago+1):
    df[ str(i)+'days_ago_temp' ]  = df[df.columns[1]].shift(i)
    df[ str(i)+'days_ago_rain' ]  = df[df.columns[2]].shift(i)
    df[ str(i)+'days_ago_cloud' ]  = df[df.columns[3]].shift(i)
    


In [82]:
df.head()

,date,temperture,rain,cloud,POSIX,year,month,day,day_of_week,1days_ago_temp,...,4days_ago_cloud,5days_ago_temp,5days_ago_rain,5days_ago_cloud,6days_ago_temp,6days_ago_rain,6days_ago_cloud,7days_ago_temp,7days_ago_rain,7days_ago_cloud
0,2010-01-01,4.8,0.0,0.0,1262304000,2010,1,1,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-02,6.3,0.0,0.0,1262390400,2010,1,2,5,4.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,5.7,0.0,6.3,1262476800,2010,1,3,6,6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,6.5,0.0,5.0,1262563200,2010,1,4,0,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-05,7.3,2.5,3.8,1262649600,2010,1,5,1,6.5,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Machine Learing

In [83]:
from sklearn.ensemble import RandomForestRegressor as RF
rf = RF()

In [84]:
N_train = round(len(df)*0.8)

In [85]:
df = df.dropna().reset_index(drop = True)
x = df.drop(['date','temperture'],axis=1 )
x_pos = x['POSIX'].values
x = x.values
y = df['temperture'].values

#x_pos = x_pos.reshape(-1,1)
#y = y.reshape(-1,1)

x_pos_train, x_pos_test = x_pos[:N_train],x_pos[N_train:]
x_train, x_test = x[:N_train],x[N_train:]
y_train, y_test = y[:N_train],y[N_train:]

rf.fit(x_train, y_train)


/Users/shibuya/.pyenv/versions/testDash/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [86]:
rf.score(x_train,y_train), rf.score(x_test,y_test)

(0.988688568960212, 0.9317719009762636)

In [87]:
df.head()

,date,temperture,rain,cloud,POSIX,year,month,day,day_of_week,1days_ago_temp,...,4days_ago_cloud,5days_ago_temp,5days_ago_rain,5days_ago_cloud,6days_ago_temp,6days_ago_rain,6days_ago_cloud,7days_ago_temp,7days_ago_rain,7days_ago_cloud
0,2010-01-08,7.6,0.0,2.0,1262908800,2010,1,8,4,7.3,...,5.0,5.7,0.0,6.3,6.3,0.0,0.0,4.8,0.0,0.0
1,2010-01-09,7.2,0.0,2.0,1262995200,2010,1,9,5,7.6,...,3.8,6.5,0.0,5.0,5.7,0.0,6.3,6.3,0.0,0.0
2,2010-01-10,7.0,0.0,0.8,1263081600,2010,1,10,6,7.2,...,0.5,7.3,2.5,3.8,6.5,0.0,5.0,5.7,0.0,6.3
3,2010-01-11,5.3,0.0,9.3,1263168000,2010,1,11,0,7.0,...,6.3,6.5,0.0,0.5,7.3,2.5,3.8,6.5,0.0,5.0
4,2010-01-12,3.9,6.5,10.0,1263254400,2010,1,12,1,5.3,...,2.0,7.3,0.0,6.3,6.5,0.0,0.5,7.3,2.5,3.8


### estat

In [88]:
estat_df = pd.read_excel('esta.xlsx')

In [89]:
#estat_df.head(15)

In [90]:
#estat_df.tail(10)

In [91]:
estat_df = estat_df[9:85].reset_index(drop = True)

In [92]:
estat_df.columns = ['date_code','date','cost']

In [93]:
estat_df['cost'] = estat_df['cost'].str.replace(',','')

In [94]:
#estat_df['cost']= pd.to_numeric(estat_df['cost'])
estat_df['date']= estat_df['date'].astype(str)
estat_df['date'] = pd.to_datetime(estat_df['date'],format = '%Y年%m月')
estat_df = estat_df.drop(['date_code'],axis = 1)

In [95]:
estat_df

,date,cost
0,2015-01-01,1526
1,2015-02-01,1808
2,2015-03-01,1712
3,2015-04-01,1515
4,2015-05-01,1874
...,...,...
71,2020-12-01,1105
72,2021-01-01,450
73,2021-02-01,682
74,2021-03-01,1421


In [102]:
df_merge = df[df['year']>= 2015 ].reset_index(drop= True)
df_merge = df[df['day'] == 1 ].reset_index(drop= True)
estat_df = estat_df[:72]
df_merge.tail()

,date,temperture,rain,cloud,POSIX,year,month,day,day_of_week,1days_ago_temp,...,4days_ago_cloud,5days_ago_temp,5days_ago_rain,5days_ago_cloud,6days_ago_temp,6days_ago_rain,6days_ago_cloud,7days_ago_temp,7days_ago_rain,7days_ago_cloud
115,2019-09-01,27.7,0.0,10.0,1567296000,2019,9,1,6,27.3,...,9.8,25.3,6.0,10.0,25.3,0.0,7.5,25.8,15.0,6.5
116,2019-10-01,24.5,0.0,5.3,1569888000,2019,10,1,1,25.2,...,5.0,22.8,0.0,7.0,24.3,0.0,7.5,25.4,1.0,6.8
117,2019-11-01,17.4,0.0,0.0,1572566400,2019,11,1,4,16.8,...,7.0,18.3,0.0,9.5,18.1,0.5,7.0,15.1,120.5,10.0
118,2019-12-01,7.4,0.0,6.8,1575158400,2019,12,1,6,7.3,...,10.0,8.8,1.5,10.0,16.6,1.0,8.0,15.9,19.0,10.0
119,2020-01-01,5.5,0.0,3.0,1577836800,2020,1,1,2,8.2,...,0.5,8.7,0.5,2.8,7.8,0.0,10.0,5.9,0.0,9.3


## RF

In [108]:
df = pd.merge(df_merge, estat_df)
df.columns

Index(['date', 'temperture', 'rain', 'cloud', 'POSIX', 'year', 'month', 'day',
       'day_of_week', '1days_ago_temp', '1days_ago_rain', '1days_ago_cloud',
       '2days_ago_temp', '2days_ago_rain', '2days_ago_cloud', '3days_ago_temp',
       '3days_ago_rain', '3days_ago_cloud', '4days_ago_temp', '4days_ago_rain',
       '4days_ago_cloud', '5days_ago_temp', '5days_ago_rain',
       '5days_ago_cloud', '6days_ago_temp', '6days_ago_rain',
       '6days_ago_cloud', '7days_ago_temp', '7days_ago_rain',
       '7days_ago_cloud', 'cost'],
      dtype='object')

In [110]:
x = df.drop(['date','cost'],axis=1 )
x = x.loc[:,['POSIX','temperture', 'rain', 'cloud', 'POSIX', 'year', 'month']]

x_pos = x['POSIX'].values
x = x.values
y = df['cost'].values

N_train =  round(len(df)*0.8)

rf = RF()
#x_pos = x_pos.reshape(-1,1)
#y = y.reshape(-1,1)

x_pos_train, x_pos_test = x_pos[:N_train],x_pos[N_train:]
x_train, x_test = x[:N_train],x[N_train:]
y_train, y_test = y[:N_train],y[N_train:]

rf.fit(x_train, y_train)

/Users/shibuya/.pyenv/versions/testDash/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [111]:
rf.score(x_train,y_train),rf.score(x_test,y_test)

(0.9083161397688648, -0.09019652642051712)